In [3]:
import pandas as pd
database=pd.read_csv('i-01',header=None,delimiter='\t')
database

,0,1,2,3,4
0,both,sedentary,moderate,yes,i100
1,both,sedentary,moderate,no,i100
2,health,sedentary,moderate,yes,i500
3,appearance,active,moderate,yes,i500
4,appearance,moderate,aggressive,yes,i500
5,appearance,moderate,aggressive,no,i100
6,health,moderate,aggressive,no,i500
7,both,active,moderate,yes,i100
8,both,moderate,aggressive,yes,i500
9,appearance,active,aggressive,yes,i500


In [8]:
class Classifer:
    def __init__(self,filename,dataFormat):
        '''
        filename  文件名
        dataFormat 数据文件格式列表
        '''
        total=0
        classes={}  #存类别出现的次数  为先验概率做准备
        counts={}   #每个类别对应的属性值出现的次数  为条件概率做准备
        self.format=dataFormat.strip().split('\t')  #表示格式  数组
        self.prior={}  #先验概率
        self.condition={}  #条件概率
        
        #构建文件名
        f=open(filename,'r')
        lines=f.readlines()
        f.close()
        for line in lines:
            total=total+1  #总记录数
            fields=line.strip().split('\t')   #切分每一条数据
            
            vector=[]   #存原始数据的位置
            for i in range( len(fields) ):
                #判断这一列是什么类型的值
                if self.format[i]=='attr':
                    vector.append(fields[i])
                elif self.format[i]=='class':
                    category=fields[i]
                    
            #记录每个类别出现的次数
            classes.setdefault(category,0)
            counts.setdefault( category,{})
            #类别数加1
            classes[category]+=1  #{'i100':1,'i500':1}
            
            #处理每条记录出现的属性值
            #循环vector 取出每列值
            col=0
            for columnValue in vector:
                col+=1
                counts[category].setdefault(col,{} )
                counts[category][col].setdefault(columnValue,0 )  #{'i100':{1:{'both':1},2:{'sedentary':1}  }  }
                counts[category][col][columnValue]+=1
                
        #对应的计算成概率        
        #1.先验概率
        for (category,count) in classes.items():
            self.prior[category]=count/total
            
        #2.条件概率   注意 colums是一个字典
        for (category,columns) in counts.items():
            self.condition.setdefault(category,{})
            for (col,valueCount) in columns.items():
                self.condition[category].setdefault(col,{})
                for (attrValue,count) in valueCount.items():
                    self.condition[category][col][attrValue] = count/classes[category]
        
        #最后将  分类数和条件数值数存到self中
        self.classCounts=classes  #分类计数
        self.conditionCount=counts  #条件计数
        
    def classify(self,itemVector):
        '''
        朴素贝叶斯：求出所有可能性的概率值，取大值
        '''
        self.result=[]  #[ (后验概率，类别1),(后验概率，类别2),.....]
        for (category,priors) in self.prior.items():
            prob=priors  #这个category类别的先验概率
            col=1
            for colValue in itemVector:
                if not colValue in self.condition[category][col]:
                    prob=0
                else:
                    prob=prob * self.condition[category][col][colValue]
                col+=1
            self.result.append( (prob,category) )  # [(0.3,'i100'), ]
        return max(self.result)[1]

In [9]:
a=Classifer('i-01','attr\tattr\tattr\tattr\tclass')
print(a.classCounts)
print(a.conditionCount)

#模型
print(a.prior)
print(a.condition)

{'i100': 6, 'i500': 9}
{'i100': {1: {'both': 3, 'appearance': 2, 'health': 1}, 2: {'sedentary': 3, 'moderate': 1, 'active': 2}, 3: {'moderate': 5, 'aggressive': 1}, 4: {'yes': 2, 'no': 4}}, 'i500': {1: {'health': 4, 'appearance': 3, 'both': 2}, 2: {'sedentary': 2, 'active': 4, 'moderate': 3}, 3: {'moderate': 3, 'aggressive': 6}, 4: {'yes': 6, 'no': 3}}}
{'i100': 0.4, 'i500': 0.6}
{'i100': {1: {'both': 0.5, 'appearance': 0.3333333333333333, 'health': 0.16666666666666666}, 2: {'sedentary': 0.5, 'moderate': 0.16666666666666666, 'active': 0.3333333333333333}, 3: {'moderate': 0.8333333333333334, 'aggressive': 0.16666666666666666}, 4: {'yes': 0.3333333333333333, 'no': 0.6666666666666666}}, 'i500': {1: {'health': 0.4444444444444444, 'appearance': 0.3333333333333333, 'both': 0.2222222222222222}, 2: {'sedentary': 0.2222222222222222, 'active': 0.4444444444444444, 'moderate': 0.3333333333333333}, 3: {'moderate': 0.3333333333333333, 'aggressive': 0.6666666666666666}, 4: {'yes': 0.6666666666666666,

In [10]:
# 健康  当前锻炼级别适中  动机中等 习惯于技术设备
print('推荐购买的类别为:',a.classify(['health','moderate','moderate','yes'] ) )
a.result

推荐购买的类别为: i500


[(0.0030864197530864196, 'i100'), (0.019753086419753083, 'i500')]